In [ ]:
%pip list | grep catboost
%pip install catboost --upgrade
%pip install pandas
%pip install scikit-learn
%pip install iterative-stratification

Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

data_path = "..\data\Tabular Dataset.csv"
pokemon_df = pd.read_csv(data_path)

columns_to_drop = ['National\nDex', 'Pokemon\nName']
columns_to_drop = [col.replace('\n', '\r\n') for col in columns_to_drop]
pokemon_df = pokemon_df.drop(columns=[col for col in columns_to_drop if col in pokemon_df.columns], errors='ignore')

categorical_cols = ['Ability I', 'Ability II', 'Hidden Ability', 'Pokedex Color', 'Egg Group I', 'Egg Group II']

type_columns = ['Type I', 'Type II']
X = pokemon_df.drop(columns=[col for col in type_columns if col in pokemon_df.columns] + ['National\nDex'], errors='ignore')
X.fillna('Unknown', inplace=True)
y = pokemon_df[type_columns].fillna('Unknown')
y['Combined'] = y.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
y_combined = y['Combined']

class_counts = y_combined.value_counts()
valid_classes = class_counts[class_counts >= 10].index
valid_indices = y_combined.isin(valid_classes)
X_valid = X[valid_indices]
y_valid = y_combined[valid_indices]

skf = StratifiedKFold(n_splits=10, shuffle=True)
fold_accuracies = []
type1_accuracies = []
type2_accuracies = []

cm_type1_list = []
cm_type2_list = []

all_y_test_type1 = []
all_y_pred_type1 = []
all_y_test_type2 = []
all_y_pred_type2 = []

for fold, (train_idx, test_idx) in enumerate(skf.split(X_valid, y_valid)):
    print(f"Training on fold {fold+1}/10")

    X_train, X_test = X_valid.iloc[train_idx], X_valid.iloc[test_idx]
    y_train, y_test = y_valid.iloc[train_idx], y_valid.iloc[test_idx]

    for col in categorical_cols:
        if col in pokemon_df.columns:
            pokemon_df[col] = pokemon_df[col].astype(str)
            X_train.loc[:, col] = X_train.loc[:, col].astype('category')  
            X_test.loc[:, col] = X_test.loc[:, col].astype('category')

    train_pool = Pool(X_train, y_train, cat_features=categorical_cols)
    test_pool = Pool(X_test, y_test, cat_features=categorical_cols)

    model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, loss_function='MultiClass', eval_metric='MultiClass')
    model.fit(train_pool, eval_set=test_pool, verbose=100, early_stopping_rounds=50)

    y_pred = model.predict(test_pool)
    y_pred_type1 = [pred[0].split('_')[0] for pred in y_pred]
    y_pred_type2 = [pred[0].split('_')[1] for pred in y_pred]
    y_test_type1 = [label.split('_')[0] for label in y_test]
    y_test_type2 = [label.split('_')[1] for label in y_test]


    y_proba = model.predict_proba(test_pool)

    type_categories = ['Normal', 'Fire', 'Water', 'Grass', 'Flying', 'Fighting', 'Poison', 'Electric', 'Ground', 'Rock', 'Psychic', 'Ice', 'Bug', 'Ghost', 'Steel', 'Dragon', 'Dark', 'Fairy']
    type1_probabilities = defaultdict(list)
    type2_probabilities = defaultdict(list)

    for i, proba in enumerate(y_proba):
      for j, class_name in enumerate(model.classes_):
          type1, type2 = class_name.split('_')
          type1_probabilities[X_test.index[i]].append((type1, proba[j]))
          type2_probabilities[X_test.index[i]].append((type2, proba[j]))

    type1_proba_df = pd.DataFrame.from_dict(type1_probabilities, orient='index')
    type1_proba_df = type1_proba_df.apply(lambda row: pd.Series({type: proba for type, proba in row}), axis=1)
    type1_proba_df = type1_proba_df.reindex(columns=type_categories, fill_value=0)

    type2_proba_df = pd.DataFrame.from_dict(type2_probabilities, orient='index')
    type2_proba_df = type2_proba_df.apply(lambda row: pd.Series({type: proba for type, proba in row}), axis=1)
    type2_proba_df = type2_proba_df.reindex(columns=type_categories, fill_value=0)

    print(f"Type I Probabilities for Fold {fold + 1}:\n{type1_proba_df}\n")
    print(f"Type II Probabilities for Fold {fold + 1}:\n{type2_proba_df}\n")

    accuracy_type1 = accuracy_score(y_test_type1, y_pred_type1)
    accuracy_type2 = accuracy_score(y_test_type2, y_pred_type2)
    type1_accuracies.append(accuracy_type1)
    type2_accuracies.append(accuracy_type2)

    precision_type1 = precision_score(y_test_type1, y_pred_type1, average='weighted')
    recall_type1 = recall_score(y_test_type1, y_pred_type1, average='weighted')
    f1_type1 = f1_score(y_test_type1, y_pred_type1, average='weighted')

    precision_type2 = precision_score(y_test_type2, y_pred_type2, average='weighted')
    recall_type2 = recall_score(y_test_type2, y_pred_type2, average='weighted')
    f1_type2 = f1_score(y_test_type2, y_pred_type2, average='weighted')

    cm_type1 = confusion_matrix(y_test_type1, y_pred_type1, labels=type_categories)
    cm_type2 = confusion_matrix(y_test_type2, y_pred_type2, labels=type_categories)
    cm_type1_list.append(cm_type1)
    cm_type2_list.append(cm_type2)

    all_y_test_type1.extend(y_test_type1)
    all_y_pred_type1.extend(y_pred_type1)
    all_y_test_type2.extend(y_test_type2)
    all_y_pred_type2.extend(y_pred_type2)

    print(f"Fold {fold+1} Type I - Precision: {precision_type1:.4f}, Recall: {recall_type1:.4f}, F1: {f1_type1:.4f}")
    print(f"Fold {fold+1} Type II - Precision: {precision_type2:.4f}, Recall: {recall_type2:.4f}, F1: {f1_type2:.4f}")
    print(f"Accuracy for Type 1: {accuracy_type1:.4f}")
    print(f"Accuracy for Type 2: {accuracy_type2:.4f}")
    fold_accuracy = accuracy_score(y_test, y_pred)
    fold_accuracies.append((accuracy_type1, accuracy_type2))
    print(f"Fold {fold+1} accuracy: {fold_accuracy:.4f}")

Remaining columns in dataset after dropping 'National Dex' and 'Pokemon Name': Index(['National\r\nDex', 'Pokemon\r\nName', 'HP', 'Attack', 'Defense',
       'Special\r\nAttack', 'Special\r\nDefense', 'Speed', 'Total',
       'Ability I', 'Ability II', 'Hidden Ability', 'Weight (kg)',
       'Height (m)', 'Pokedex Color', 'Egg Group I', 'Egg Group II', 'Type I',
       'Type II'],
      dtype='object')
Data types of X_train columns before Pool: National\r\nDex       float64
Pokemon\r\nName        object
HP                      int64
Attack                  int64
Defense                 int64
Special\r\nAttack       int64
Special\r\nDefense      int64
Speed                   int64
Total                   int64
Ability I              object
Ability II             object
Hidden Ability         object
Weight (kg)           float64
Height (m)            float64
Pokedex Color          object
Egg Group I            object
Egg Group II           object
dtype: object


C:\Users\maxfe\AppData\Local\Temp\ipykernel_110936\1199800706.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna('Unknown', inplace=True)
C:\Users\maxfe\AppData\Local\Temp\ipykernel_110936\1199800706.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Combined'] = y.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
C:\Users\maxfe\AppData\Local\Temp\ipykernel_110936\1199800706.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=1]="Pelipper": Cannot convert 'b'Pelipper'' to float

In [ ]:
type_categories = ['Normal', 'Fire', 'Water', 'Grass', 'Flying', 'Fighting', 'Poison', 
                  'Electric', 'Ground', 'Rock', 'Psychic', 'Ice', 'Bug', 'Ghost', 
                  'Steel', 'Dragon', 'Dark', 'Fairy', 'Unknown']  
type_categories_type1 = sorted(list(set(all_y_test_type1) | set(all_y_pred_type1)))
type_categories_type2 = sorted(list(set(all_y_test_type2) | set(all_y_pred_type2)))

all_possible_type2_classes = sorted(list(set(all_y_test_type2) | set(all_y_pred_type2) | set(type_categories)))

avg_cm_type1 = confusion_matrix(all_y_test_type1, all_y_pred_type1, labels=type_categories_type1)
avg_cm_type2 = confusion_matrix(all_y_test_type2, all_y_pred_type2, labels=all_possible_type2_classes)

avg_acc_type1 = sum(type1_accuracies) / len(type1_accuracies)
avg_acc_type2 = sum(type2_accuracies) / len(type2_accuracies)

disp_type1 = ConfusionMatrixDisplay(confusion_matrix=avg_cm_type1, display_labels=type_categories_type1)
disp_type2 = ConfusionMatrixDisplay(confusion_matrix=avg_cm_type2, display_labels=all_possible_type2_classes)

y_pred_combined = ['_'.join([pred1, pred2]) for pred1, pred2 in zip(y_pred_type1, y_pred_type2)]
total_acc = accuracy_score(y_test, y_pred_combined)

print("\nClassification Report for Type I (All Folds):")
print(classification_report(all_y_test_type1, all_y_pred_type1, target_names=type_categories_type1, zero_division=0))

print("\nClassification Report for Type II (All Folds):")
print(classification_report(all_y_test_type2, all_y_pred_type2, labels=all_possible_type2_classes, target_names=all_possible_type2_classes, zero_division=0)) 

print(f"\nAvg Type I Acc: {avg_acc_type1:.4f}")
print(f"\nAvg Type II Acc: {avg_acc_type2:.4f}")
print(f"\nAvg Total acc: {total_acc:.4f}")

disp_type1.plot(cmap=plt.cm.Blues)
plt.title('Type I Confusion Matrix')
plt.show()

disp_type2.plot(cmap=plt.cm.Blues)
plt.title('Type II Confusion Matrix')
plt.show()
